In [13]:
import pandas as pd
from sqlalchemy import create_engine

df_cleaned = pd.read_csv("data/processed/superstore_cleaned.csv", parse_dates=["Order Date", "Ship Date"])

engine = create_engine("sqlite:///data/superstore.db", echo=True)

df_cleaned.to_sql("raw_sales", engine, if_exists="replace", index=False)


2025-05-19 19:05:45,601 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:05:45,608 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw_sales")
2025-05-19 19:05:45,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:05:45,617 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw_sales")
2025-05-19 19:05:45,618 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:05:45,620 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-19 19:05:45,622 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:05:45,627 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-19 19:05:45,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:05:45,632 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("raw_sales")
2025-05-19 19:05:45,634 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-0

9994

In [47]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text(
    
"""
CREATE TABLE IF NOT EXISTS dim_product AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales;

CREATE TABLE IF NOT EXISTS dim_product AS
    SELECT DISTINCT
      Category,
      "Sub-Category" AS sub_category

FROM raw_sales
   """         
                   )
                )

2025-05-19 19:52:14,420 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:52:14,421 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS dim_product AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales;

CREATE TABLE IF NOT EXISTS dim_product AS
    SELECT DISTINCT
      Category,
      "Sub-Category" AS sub_category

FROM raw_sales
   
2025-05-19 19:52:14,423 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ()
2025-05-19 19:52:14,424 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (sqlite3.ProgrammingError) You can only execute one statement at a time.
[SQL: 
CREATE TABLE IF NOT EXISTS dim_product AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales;

CREATE TABLE IF NOT EXISTS dim_product AS
    SELECT DISTINCT
      Category,
      "Sub-Category" AS sub_category

FROM raw_sales
   ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [37]:
with engine.begin() as conn:

    conn.execute(text
        ("""
        CREATE TABLE IF NOT EXISTS facts_sales AS
        SELECT 
            ROW_NUMBER() OVER (ORDER by "ORDER ID") AS sale_id,
            DATE("Order Date")                      AS order_date,
            Category,
            "Sub-Category" AS sub_category,
            Region,
            Sales,
            Quantity,
            Profit,
            CAST((JULIANDAY("Ship Date") - JULIANDAY("Order Date")) AS INTEGER) AS shipping_time_days
    FROM raw_sales;


    
      """)
            )

2025-05-19 19:33:31,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:33:31,980 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS facts_sales AS
        SELECT 
            ROW_NUMBER() OVER (ORDER by "ORDER ID") AS sale_id,
            DATE("Order Date")                      AS order_date,
            Category,
            "Sub-Category" AS sub_category,
            Region,
            Sales,
            Quantity,
            Profit,
            CAST((JULIANDAY("Ship Date") - JULIANDAY("Order Date")) AS INTEGER) AS shipping_time_days
    FROM raw_sales;


    
      
2025-05-19 19:33:31,980 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ()
2025-05-19 19:33:32,032 INFO sqlalchemy.engine.Engine COMMIT


In [43]:
pd.read_sql("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION", engine)

pd.read_sql("SELECT d.quater, COUNT(*) AS num_orders FROM facts_sales f JOIN dim_date d USING(order_date) GROUP BY d.quarter", engine)

pd.read_sql("""SELECT sub_category, SUM(Profit) AS total_profit
            FROM facts_sales
            GROUP BY sub_category
            ORDER BY total_profit DESC LIMIT 5""", engine)




2025-05-19 19:48:14,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:48:14,462 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION")
2025-05-19 19:48:14,462 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:48:14,462 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION")
2025-05-19 19:48:14,465 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:48:14,465 INFO sqlalchemy.engine.Engine SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION
2025-05-19 19:48:14,467 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 19:48:14,471 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-19 19:48:14,472 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:48:14,473 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT d.quater, COUNT(*) AS num_orders FROM facts_sales f JOIN dim_date d USING

OperationalError: (sqlite3.OperationalError) no such table: dim_date
[SQL: SELECT d.quater, COUNT(*) AS num_orders FROM facts_sales f JOIN dim_date d USING(order_date) GROUP BY d.quarter]
(Background on this error at: https://sqlalche.me/e/20/e3q8)